## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [25]:
import pandas as pd 
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [26]:
data = pd.read_csv('/datasets/data.csv')
print(data.info()) #изучаем информацию о структуре полученных данных
print(data.head()) #смотрим, как данные выглядят

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1  

In [27]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [28]:
print(data['days_employed'].value_counts())

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64


In [29]:
print(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


In [30]:
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In [31]:
print(data['education_id'].value_counts())

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


In [32]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [33]:
print(data['family_status_id'].value_counts())

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


In [34]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [35]:
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


In [36]:
print(data['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


In [37]:
print(data['total_income'].value_counts())

112874.418757    1
133912.272223    1
182036.676828    1
122421.963500    1
198271.837248    1
                ..
133299.194693    1
115080.782380    1
84896.781597     1
153838.839212    1
150014.128510    1
Name: total_income, Length: 19351, dtype: int64


In [38]:
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Исходя из первичного анализа полученных данных видно, что данные приготовлены плохо:  в некоторых столбцах данные не приведены к единому формату, в некоторых столбцах содержатся откровенно бессмысленные значения и т.д. Так что предстоит много работы.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [39]:
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median())) #замена пропущенных значений
data['days_employed'].abs()
data['days_employed'] = data.groupby('income_type')['days_employed'].transform(lambda y: y.fillna(y.mean())) 
print(data[data['total_income'].isnull()].count()) #проверка заполненности пропущенных значений
print(data[data['days_employed'].isnull()].count())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Вывод

Пропущенные значения содержатся в столбцах days_employed и total_incom. Причины, по которой произошли эти потери, могут быть различны: сбой в системе, невнимательность сотрудников. В столбце days_employd заменим пропущенные значения на среднее (и избавимся от минусов). В столбце total_incom заменим пропущенные значения на медианные (это более правильно, чем замена средним). Но стоит обратиться к сотрудникам банка, чтобы на будущее подобных пропусков не было.

### Замена типа данных

In [40]:
data['total_income'] = data['total_income'].astype('int')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
None


### Вывод

С помощью метода astype переводим данные столбца total_incom в целочисленный формат, поскольку для дальнейшего анализа нам не нужна такая точность, а перевод упростит вычисления.

### Обработка дубликатов

In [41]:
data['education'] = data['education'].str.lower() #приводим все к одному виду 
data['children'] = data['children'].replace(-1,1)
print('Всего дубликатов',data.duplicated().sum()) #смотрим, сколько дубликатов в данных
data = data.drop_duplicates().reset_index(drop=True) #удаляем дубликаты
print('Всего дубликатов',data.duplicated().sum()) #проверяем, что все удалилось
print(data['children'].value_counts()) #проверка данных
print(data['education'].value_counts()) #проверка данных

Всего дубликатов 71
Всего дубликатов 0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521    False
21522    False
21523    False
21524    False
Length: 21525, dtype: bool
Всего дубликатов 0
0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64
среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


### Вывод

В столбце children были продублированы строки с 1 ребенком. Необходимо добавить эти значения к тем строкам, где просто один ребенок. В столбце education так же есть продублированные значения в разных регистрах, поэтому приведем сначала все данные к одному виду. Затем применим инструменты поиска и удаления дубликатов pandas. Причины возникновения дубликатов могут быть различными: неправильное соединение данных из разных источников, ошибки пользователя при занесении информации. Как можно видеть, после приведения даннных к одному виду выявился 71 дубликат, что приведет к неверным результатам при дальнейшем анализе, поэтому их необходимо удалить.

### Лемматизация

In [18]:
all_lemmas = [] #создаем пустой массив для всех лемм
for purpose in data['purpose']:
    purpose_lemmas = m.lemmatize(purpose)
    all_lemmas += purpose_lemmas #заполняем его
print(Counter(all_lemmas)) #находим уникальные значения и вручную вносим их в новый массив
category = ['недвижимость','жилье','свадьба', 'образование','автомобиль'] #выделенные уникальные слова
def lemma(series):                                              #функция для лемматизации столбца
    for i in range (len(series)):
        lem = m.lemmatize(series[i])
        for j in category:
            if j in lem:
                series[i] = j
    return series
data['purpose'] = lemma(data['purpose'])
data['purpose'] = data['purpose'].replace('жилье', 'недвижимость')
print(data['purpose'].value_counts()) #проверка, что ничего не потеряли

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64


### Вывод

Для лемматизации были выведены уникальные значения в начале анализа и был вручную составлен список уникальных слов. К сожалению, автоматизировать процесс выбора нужных лемм невозможен, так как требуется контекст (а его мы узнали в начале анализа, когда выводили содержимое столбцов). Поэтому список нужных лемм формируем вручную. Затем с помощью библиотеки pymystem3 была проведена лемматизация столбца purpose.

### Категоризация данных

In [19]:
def children_count(children): #категоризация по количеству детей
        if children < 1:
                return 'бездетные'
        if children > 2:
                return 'многодетные'
        return '1-2 ребенка'
data['child_category'] = data['children'].apply(children_count)
print(data['child_category'].value_counts())

бездетные      14091
1-2 ребенка     6907
многодетные      456
Name: child_category, dtype: int64


In [20]:
print(data['total_income'].describe()) #рассчитаем квартили
def disq_incom(total_income): #категоризация по уровню дохода
        if total_income <= 107798:
                return 'бедные'
        if 107798 < total_income <= 142594:
                return 'средний класс'
        if 142594 < total_income <= 195549:
                return 'богатые'
        return 'очень богатые'
data['income_category'] = data['total_income'].apply(disq_incom)
print(data['income_category'].value_counts())

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64
средний класс    5461
бедные           5382
очень богатые    5381
богатые          5230
Name: income_category, dtype: int64


### Вывод

Данные категоризируем по следующим словарям: количество детей в семье, уровень заработка относительно среднего. Семьи по количеству детей разделим так же, как это принято в государстве: многодетные это 3 или более ребенка. Для категоризации по уровню дохода рассчитаем квартили (поскольку мы разделим на 4 равные группы по количеству) и исходя из этого проведем деление.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [21]:
def credit_count(series):
    return "{0:.2%}".format(series.sum()/series.count())
data_pivot = data.pivot_table(index = 'child_category', values = 'debt', aggfunc = credit_count)
print(data_pivot.sort_values(by = 'debt'))

                 debt
child_category       
бездетные       7.54%
многодетные     8.55%
1-2 ребенка     9.25%


### Вывод

Исходя из проведенного анализа можно сделать вывод, что чаще всего кредит возвращают бездетные пары, а реже всего семьи с 1-2 детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
data_pivot = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = credit_count)
print(data_pivot.sort_values(by = 'debt'))

                        debt
family_status               
вдовец / вдова         6.57%
в разводе              7.11%
женат / замужем        7.55%
гражданский брак       9.35%
Не женат / не замужем  9.75%


### Вывод

Из данной таблицы видно, что чаще всего возвращают кредит в срок люди, состоявшие в отношениях, но потерявшие мужа/жену. Тогда как люди, не имеющие официальных отношений, просрачивают выплаты с большей вероятностью.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
data_pivot = data.pivot_table(index = 'income_category', values = 'debt', aggfunc = credit_count)
print(data_pivot.sort_values(by = 'debt'))

                  debt
income_category       
очень богатые    7.17%
бедные           7.93%
богатые          8.51%
средний класс    8.84%


### Вывод

Из анализа видно, что вероятнее всего просрочат кредит средний класс и богатые (2 близких финансово класса), тогда как бедный класс и самые богатые (классы на разных полюсах по финансам) с большей вероятностью вернут кредит.

- Как разные цели кредита влияют на его возврат в срок?

In [24]:
data_pivot = data.pivot_table(index = 'purpose', values = 'debt', aggfunc = credit_count)
print(data_pivot.sort_values(by = 'debt'))

               debt
purpose            
недвижимость  7.23%
свадьба       8.00%
образование   9.22%
автомобиль    9.36%


### Вывод

Из данной таблицы видно, что зависимость между целью кредита и возвратом сдедующая: люди, берущие кредит на недвижимость чаще всего возвращают долг, а те, кто берут кредит на машину и образование, реже всего.

### Шаг 4. Общий вывод

После проведенного анализа можно сформулировать следующие выводы:
#1) Количество детей в семье довольно логично влияет на возможность погашения: бездетные семьи выплачивают чаще, чем многодетные. Однако семьи, у которых 1-2 ребенка просрачивают чаще, чем многодетные. Это контринтуитивно. 
#2) По финансовому положению так же не все очевидно: люди, которые находятся примерно на одном уровне дохода(средние и богатые) не выплачивают кредит с большей вероятностью, чем люди, доходы которых разительно отличаются (бедные и очень богатые).
#3) Цели кредита и семейное положение влияют на срок погашения : больше всего просрачивают кредит те, кто берут деньги на автомобиль и образование. Вдовы и вдовцы выплачивают чаще всех, а люди, не обремененные семьей, больше всех просрачивают выплаты.
Но в любом случае, это весьма поверхностный анализ, поэтому некоторые выводы оказались неожиданными. Для выявления более четкой зависимости требуется более глубокая аналитика.